pretendo usar um decoder para tratar das soluções que não são válidas.

The order crossover operator [92] was designed by Davis for order-based permutation problems. It begins in a similar fashion to PMX, by copying a
randomly chosen segment of the first parent into the offspring. However, it
proceeds differently because the intention is to transmit information about
relative order from the second parent.
1. Choose two crossover points at random, and copy the segment between
them from the first parent (PI) into the first offspring.
2. Starting from the second crossover point in the second parent, copy the
remaining unused numbers into the first child in the order that they appear
in the second parent, wrapping around at the end of the list.
3. Create the second offspring in an analogous manner, with the parent roles
reversed. 

In [1]:
import random

In [ ]:
class Operation:
    def __init__(self, id, machine, processing_time):
        self.id = id
        self.machine = {machine}  # Set containing the single machine this operation can use
        self.processing_times = {machine: processing_time}

    def able(self, machine):
        return machine in self.machine

    def get_processing_time(self, machine):
        return self.processing_times.get(machine, 0)

class Job:
    def __init__(self, id, operations):
        self.id = id
        self.operations = operations  # List of Operation objects in order of precedence

class JobShopSolution:
    def __init__(self, jobs, machines, genotype):
        self.genotype = genotype
        self.jobs = jobs
        self.machines = machines
        #phenotype = schedule

        self.fitness, self.phenotype = self.build_schedule()

    def get_fitness(self):
        return self.fitness

    def build_schedule(self):
		# Dictionary to track when each machine will be free
        machine_end_times = {machine: 0 for machine in self.machines}
		# Dictionary to track when each job's previous operation finished
        job_end_times = {job.id: 0 for job in self.jobs}

        schedule = [] #guarda o job, machine e tempor de inicio e fim
        #self.genotype = [O1,O2,O3,O4]
        #O1 and O3 BELONG TO J1 (TEMQ UE RESPEITAR PRECEDENCIA)
        #O2 AND O4 BELONG TO J2 (TEM QUE RESPEITAR PRECEDENCIA)
        operation_by_job = {} #guarda os jobs de cada operacao

        for job in self.jobs:
            for operation in job.operations:#itera pelas operacoes de cada job
                operation_by_job[operation] = job.id #guarda o job de cada operacao, vai ser usado para saber se a operacao esta na ordem certa

        for operation in self.genotype:#self.genotype = [O1,O2,O3,O4]
            job_id = operation_by_job[operation]
            machine = operation.machine #operacao eh colocada na maquina que ela pode usar
            processing_time = operation.get_processing_time(machine)

			#tempo de inicio da operacao
            start_time = max(machine_end_times[machine], job_end_times[job_id])
            #eh o maior entre o tempo em que a maquina ficou livre ou que a operacao anterior terminou
            #ou seja, pode trabalhar quando a maquina aficar livre, ou quando a operacao anterior terminou
            end_time = start_time + processing_time #tempo de fim da operacao

            machine_end_times[machine] = end_time # Machine will be busy until end_time
            job_end_times[job_id] = end_time # Job can't start next operation until end_time

			#adiciona a operacao no schedule
            schedule.append((operation, machine, start_time, end_time))

            fitness = max(machine_end_times.values()) #tempo de execucao do ultimo job
            return fitness, schedule

In [ ]:
def initialize_population(population_size, jobs):
	population =[]

	for _ in range(population_size):
		chromossome = []
		available_operations = []
		completed_operations = set()#operacoes ja completadas do cromossomo especifico

		for job in jobs:
			if job.operations:
				#adiciona apenas a primeira operacao do job para permitir apenas entradas validas
				available_operations.append(job.operations[0])

		while available_operations:

			operation = random.choice(available_operations)
			chromossome.append(operation)
			available_operations.remove(operation)
			completed_operations.add(operation)

			for job in jobs:
				if operation in job.operations:
					op_index = job.operations.index(operation)
					#se ha uma proxima operacao nesse job
					if op_index + 1 < len(job.operations):
						next_operation = job.operations[op_index]

						#se todos os predecessores da proxima operacao ja foram completados
						if all(pred in completed_operations for pred in job.operations[:op_index + 1]):
							available_operations.append(next_operation)

		population.append(chromossome)

	return population

def tournament(population, k):
	individuals = random.sample(population, k)
	return max(individuals, key=lambda x : x.get_fitness())

#Crossover: Order-based crossover (keeps valid permutations).
#acompanha os constraints de ordem entre as operações
def order_based_crossover(parent1,parent2):
	length = len(parent1)

	point1 = random.randint(0,length-2)
	point2 = random.randint(point1+1, length-1)

	offspring1 = [None]

